In [6]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.data_structures import mine_CARs

In [9]:
from pyarc.qcba.transformation import QCBATransformation

from pyarc import CBA
from pyarc.data_structures import TransactionDB
from pyarc.qcba.data_structures import QuantitativeDataFrame
import pandas as pd
from pyarc.qcba import QCBA

from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

from pyarc.qcba.classifier import QuantitativeClassifier
from sklearn.metrics import accuracy_score

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

QuantitativeCAR.interval_reader = interval_reader

In [10]:
data_train_disc = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/iris0.csv")
data_train_undisc = pd.read_csv("c:/code/python/machine_learning/assoc_rules/folds_undiscr/train/iris0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/iris0.csv")

txns_train = TransactionDB.from_DataFrame(data_train_disc)
txns_test = TransactionDB.from_DataFrame(data_test)

In [11]:
quant_dataframe_train_disc = QuantitativeDataFrame(data_train_disc)
quant_dataframe_train_undisc = QuantitativeDataFrame(data_train_undisc)

In [12]:
cars = mine_CARs(data_train_disc, 50, sample=True)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=5
Rule count: 96, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=5
Rule count: 166, Iteration: 2
Increasing maxlen 5
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000

In [13]:
ids = IDS()
ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train_disc, debug=False)

cover cache prepared
overlap cache prepared


In [14]:
cba = CBA()
cba.fit(txns_train)
cba.rule_model_accuracy(txns_train)

0.9407407407407408

In [15]:
cars = [ r.car for r in ids.clf.rules ]

In [39]:
from pyids.data_structures import IDSRuleSet
import xml.etree.ElementTree as ET

ids_rules = ids.clf.rules
ids_ruleset = IDSRuleSet(ids_rules)

def indent(elem, level=0):
    i = "\n" + level*"  "
    j = "\n" + (level-1)*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for subelem in elem:
            indent(subelem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = j
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = j
    return elem     

r0 = ids_rules[0]
ruleml = r0.to_ruleml_xml()

indent(ruleml)

ET.dump(ruleml)

<Implies>
  <head>
    <Atom>
      <Var>class</Var>
    <Rel>Iris-virginica</Rel>
    </Atom>
  </head>
<body>
    <Atom>
      <Var>name</Var>
    <Rel>petallength</Rel>
    </Atom>
  <Atom>
      <Var>value</Var>
    <Rel>4.75_to_inf</Rel>
    </Atom>
  <Atom>
      <Var>name</Var>
    <Rel>petalwidth</Rel>
    </Atom>
  <Atom>
      <Var>value</Var>
    <Rel>1.75_to_inf</Rel>
    </Atom>
  <Atom>
      <Var>name</Var>
    <Rel>sepalwidth</Rel>
    </Atom>
  <Atom>
      <Var>value</Var>
    <Rel>2.95_to_3.35</Rel>
    </Atom>
  </body>
</Implies>


In [16]:
qcba = QCBA(quant_dataframe_train_undisc, rules=cars)
qcba.fit()

qcba_cba = QCBA(quant_dataframe_train_undisc, cba_rule_model=cba)
qcba_cba.fit()

applying selected transformations
refitting
literal pruning
trimming
extending
[                                                  ]
[###                                               ]
[######                                            ]
[#########                                         ]


KeyboardInterrupt: 

In [10]:
print("Přesnost IDS:", ids.score(quant_dataframe_train_disc))
print("Přesnost CBA:", cba.rule_model_accuracy(txns_train))
print("Přesnost IDS-QCBA:", qcba.score(quant_dataframe_train_undisc))
print("Přesnost CBA-QCBA:", qcba_cba.score(quant_dataframe_train_undisc))

Přesnost IDS: 0.6148148148148148
Přesnost CBA: 0.9407407407407408
Přesnost IDS-QCBA: 0.9777777777777777
Přesnost CBA-QCBA: 0.9629629629629629


In [11]:
print("Délka IDS solution setu:", len(ids.clf.rules))
print("Délka IDS-QCBA solution setu:", len(qcba.clf.rules))

Délka IDS solution setu: 20
Délka IDS-QCBA solution setu: 5
